In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models import vgg16
from torchvision.models.detection import fasterrcnn_resnet50_fpn

class FastRCNN(nn.Module):
    def __init__(self, num_classes):
        super(FastRCNN, self).__init__()
        self.feature_extractor = vgg16(pretrained=True).features
        self.roi_pooling = nn.AdaptiveMaxPool2d((7, 7))
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Linear(4096, num_classes + 1)  # +1 for background class
        )

    def forward(self, images, boxes):
        features = self.feature_extractor(images)
        rois = self.roi_pooling(features, boxes)
        rois = rois.view(rois.size(0), -1)
        output = self.classifier(rois)
        return output

# Ejemplo de uso
num_classes = 20  # Número de clases en el conjunto de datos
model = FastRCNN(num_classes)

# Entrenamiento
images = torch.randn(4, 3, 224, 224)  # Batch de imágenes de entrada
boxes = torch.tensor([[10, 20, 50, 80], [30, 40, 60, 90], [50, 60, 80, 120], [70, 80, 100, 160]])  # Ejemplo de cuadro delimitador (x1, y1, x2, y2)
labels = torch.tensor([1, 2, 3, 4])  # Etiquetas correspondientes a los cuadros delimitadores
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Propagación hacia adelante y hacia atrás
outputs = model(images, boxes)
loss = F.cross_entropy(outputs, labels)
optimizer.zero_grad()
loss.backward()
optimizer.step()


c:\Users\Usuario\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Usuario\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to C:\Users\Usuario/.cache\torch\hub\checkpoints\vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

TypeError: AdaptiveMaxPool2d.forward() takes 2 positional arguments but 3 were given